In [4]:
# Create file directory if didn't exist
import os

input_folder = r"data"
output_folder = r"output"

try:
    os.makedirs(input_folder, exist_ok=True)
    os.makedirs(output_folder, exist_ok=True)
except OSError:
    pass

In [5]:
import os

video_list = os.listdir(input_folder)
video_list

['第01話.ts', '第02話.ts', '第03話.ts']

In [47]:
import subprocess

def convert_video(data_path:str, output_path:str, target_format:str):
    try:
        subprocess.run(["FFmpeg", "-i", data_path, "-codec", "copy", 
                        output_path], check=True)
    except Exception as e:
        print(f"Failed to convert file to {target_format} with exception:\n{e}")

In [45]:
import os

def convert_data_by_part(data_folder:str, output_folder:str, start_index:int, end_index:int, file_list, target_format:str="mp4"):
    total = 0
    for n in range(start_index, end_index):
        file = file_list[n]
        name, ext = os.path.splitext(file)
        data_path = os.path.join(data_folder, file)
        output_path = os.path.join(output_folder, f"{name}.{target_format}")
        convert_video(data_path, output_path, target_format)
        total += 1
    print(f"total image processed = {total}")

In [29]:
import os

def clear_output_folder(output_folder:str):
    files = os.listdir(output_folder)
    for file in files:
        os.remove(file)

In [34]:
import os
import threading
import multiprocessing
import math
import time

def start_convert_video(data_folder:str, output_folder:str, target_format:str, use_thread=False):
    start_time = time.time()
    video_list = os.listdir(data_folder)
    clear_output_folder(output_folder)
    
    length = len(video_list)
    if length == 0:
        print("There is no data to convert")
        return
    
    total_thread = 10
    
    if use_thread:
        threads = []
        if total_thread > length:
            total_thread = length
        divider = length//total_thread

        total = total_thread
        if length%divider != 0:
            total += 1
        for n in range(total):
            start_index = n*divider
            if start_index > length:
                continue
            end_index = min((n+1)*divider, length)
            args = data_folder, output_folder, start_index, end_index, video_list, target_format
            thread = threading.Thread(target=convert_data_by_part, args=args, daemon=True)
            threads.append(thread)
            thread.start()

        for thread in threads:
            thread.join()
    else:
        convert_data_by_part(data_folder, output_folder, 0, length, file_list)

    elapsed_time = int(time.time() - start_time)
    hour = elapsed_time//3600
    minute = (elapsed_time - hour*3600)//60
    second = (elapsed_time - hour*3600 - minute*60)
    print(f"time elapsed: {hour:02d}:{minute:02d}:{second:02d}")

In [48]:
start_convert_video(input_folder, output_folder, "mp4", use_thread=True)

total image processed = 1
total image processed = 1
total image processed = 1
time elapsed: 00:00:00
